In [1]:
from numpy import array
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
import pandas as pd
from sodapy import Socrata

In [ ]:
#Uncomment the below to open data source, download sensor data, and store it as a csv locally.

##Function to get Sensor count history data
# def sensor_count():
#     client = Socrata('data.melbourne.vic.gov.au', 'nlPM0PQJSjzCsbVqntjPvjB1f', None)
#     sensor_data_id = "b2ak-trbp"
#     results = client.get(sensor_data_id, limit=5000000)
#     df = pd.DataFrame.from_records(results)
#     df = df[['date_time', 'year', 'month', 'mdate', 'day', 'time', 'sensor_id', 'sensor_name', 'hourly_counts']]
#     return df

# sensor_history = sensor_count()

# sensor_history.to_csv('sensor_history.csv', index=False)

In [2]:
sensor_history = pd.read_csv('sensor_history.csv')

In [3]:
single_sensor = sensor_history[sensor_history.sensor_id == 4]

In [4]:
single_sensor

,date_time,year,month,mdate,day,time,sensor_id,sensor_name,hourly_counts
26,2019-11-01T18:00:00.000,2019,November,1,Friday,18,4,Town Hall (West),2950
81,2019-11-01T19:00:00.000,2019,November,1,Friday,19,4,Town Hall (West),2345
112,2022-04-05T15:00:00.000,2022,April,5,Tuesday,15,4,Town Hall (West),2132
137,2019-11-01T20:00:00.000,2019,November,1,Friday,20,4,Town Hall (West),2051
193,2019-11-01T21:00:00.000,2019,November,1,Friday,21,4,Town Hall (West),1715
...,...,...,...,...,...,...,...,...,...
4230766,2022-04-30T19:00:00.000,2022,April,30,Saturday,19,4,Town Hall (West),1641
4230767,2022-04-30T20:00:00.000,2022,April,30,Saturday,20,4,Town Hall (West),1239
4230768,2022-04-30T21:00:00.000,2022,April,30,Saturday,21,4,Town Hall (West),1083
4230769,2022-04-30T22:00:00.000,2022,April,30,Saturday,22,4,Town Hall (West),883


In [5]:
sensor_seq = []
for i in range(0, len(single_sensor)):
    sensor_seq.append(single_sensor.hourly_counts.iloc[i])

In [6]:
# split a univariate sequence
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# choose a number of time steps
n_steps = 7
# split into samples
X, y = split_sequence(sensor_seq, n_steps)


In [7]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))


In [8]:
epochs = 50
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.8

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
# model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
# model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
# model.add(LSTM(50, activation='relu'))
model.add(Dense(1))

opt = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(optimizer=opt, loss='mse', metrics = ['accuracy'])


In [9]:
# fit model
model.fit(X, y, epochs=200, verbose=1)


Epoch 1/200
3368/3368 [==============================] - 12s 3ms/step - loss: 153616.4219 - accuracy: 3.4331e-04
Epoch 2/200
3368/3368 [==============================] - 12s 3ms/step - loss: 106408.7891 - accuracy: 3.9899e-04
Epoch 3/200
3368/3368 [==============================] - 11s 3ms/step - loss: 87559.7656 - accuracy: 3.8971e-04
Epoch 4/200
3368/3368 [==============================] - 11s 3ms/step - loss: 82669.0781 - accuracy: 3.8971e-04
Epoch 5/200
3368/3368 [==============================] - 11s 3ms/step - loss: 111799.8594 - accuracy: 3.8971e-04
Epoch 6/200
3368/3368 [==============================] - 11s 3ms/step - loss: 98674.7109 - accuracy: 3.3404e-04
Epoch 7/200
3368/3368 [==============================] - 11s 3ms/step - loss: 89371.2734 - accuracy: 3.3404e-04
Epoch 8/200
3368/3368 [==============================] - 11s 3ms/step - loss: 86347.2969 - accuracy: 3.0620e-04
Epoch 9/200
3368/3368 [==============================] - 11s 3ms/step - loss: 220156.7969 - accuracy:

3368/3368 [==============================] - 11s 3ms/step - loss: 114276.3281 - accuracy: 3.7115e-04
Epoch 74/200
3368/3368 [==============================] - 11s 3ms/step - loss: 89797.5312 - accuracy: 3.8971e-04
Epoch 75/200
3368/3368 [==============================] - 11s 3ms/step - loss: 105801.1719 - accuracy: 3.8971e-04
Epoch 76/200
3368/3368 [==============================] - 11s 3ms/step - loss: 105017.3281 - accuracy: 3.8043e-04
Epoch 77/200
3368/3368 [==============================] - 11s 3ms/step - loss: 89787.5234 - accuracy: 3.8971e-04
Epoch 78/200
3368/3368 [==============================] - 11s 3ms/step - loss: 116461.3125 - accuracy: 3.8971e-04
Epoch 79/200
3368/3368 [==============================] - 11s 3ms/step - loss: 94364.8281 - accuracy: 3.8043e-04
Epoch 80/200
3368/3368 [==============================] - 11s 3ms/step - loss: 92337.2188 - accuracy: 3.8971e-04
Epoch 81/200
3368/3368 [==============================] - 11s 3ms/step - loss: 95679.9375 - accuracy: 3.8

Epoch 145/200
3368/3368 [==============================] - 11s 3ms/step - loss: 76857.4688 - accuracy: 3.8971e-04
Epoch 146/200
3368/3368 [==============================] - 11s 3ms/step - loss: 86343.4219 - accuracy: 3.8043e-04
Epoch 147/200
3368/3368 [==============================] - 11s 3ms/step - loss: 81096.9453 - accuracy: 3.8971e-04 0s - los
Epoch 148/200
3368/3368 [==============================] - 11s 3ms/step - loss: 80592.0547 - accuracy: 3.8043e-04
Epoch 149/200
3368/3368 [==============================] - 11s 3ms/step - loss: 88068.4922 - accuracy: 3.8971e-04
Epoch 150/200
3368/3368 [==============================] - 11s 3ms/step - loss: 90347.4688 - accuracy: 3.8971e-04
Epoch 151/200
3368/3368 [==============================] - 11s 3ms/step - loss: 97476.6953 - accuracy: 3.8043e-04
Epoch 152/200
3368/3368 [==============================] - 11s 3ms/step - loss: 84994.4062 - accuracy: 3.8971e-04
Epoch 153/200
3368/3368 [==============================] - 11s 3ms/step - loss:

In [10]:
# demonstrate prediction
x_input = array(sensor_seq[1:8])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print("predicted next value: ", int(yhat))
print("actual next value: ", sensor_seq[8])

predicted next value:  761
actual next value:  375


In [11]:
# demonstrate prediction
x_input = array(sensor_seq[21:28])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print("predicted next value: ", int(yhat))
print("actual next value: ", sensor_seq[28])

predicted next value:  1423
actual next value:  1379
